Avances LAB04 


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import warnings
warnings.filterwarnings('ignore')

In [7]:
def calcular_hopkins(X):
    n, d = X.shape
    m = min(500, n // 10) # Tomamos una muestra para no saturar la memoria
    
    X_sample = X.sample(m, random_state=42)
    X_uniform = np.random.uniform(X.min(axis=0), X.max(axis=0), (m, d))
    
    nbrs = NearestNeighbors(n_neighbors=2).fit(X)
    
    # Distancias de los puntos aleatorios (u)
    u_distances, _ = nbrs.kneighbors(X_uniform, n_neighbors=1)
    u_sum = np.sum(u_distances**2)
    
    # Distancias de la muestra real (w). n_neighbors=2 porque el index 0 es el mismo punto
    w_distances, _ = nbrs.kneighbors(X_sample, n_neighbors=2)
    w_sum = np.sum(w_distances[:, 1]**2)
    
    H = u_sum / (u_sum + w_sum)
    return H

In [8]:
path = "movies_2026.csv"
df = pd.read_csv(path)

# Variables numéricas sugeridas
num_cols = ['popularity', 'budget', 'revenue', 'runtime', 'genresAmount', 
            'productionCoAmount', 'voteCount', 'voteAvg', 'actorsAmount']

ejemplo_num = df[num_cols].dropna()
print(f"Dimensiones de los datos limpios: {ejemplo_num.shape}\n")

Dimensiones de los datos limpios: (19883, 9)

